**Навигация по уроку**

1. [Первое знакомство с AutoML](https://colab.research.google.com/drive/1bCWyzlp1-tcvt7TE60m4hFnRd5AWscWY)
2. [Гиперпараметры и оптимизация моделей](https://colab.research.google.com/drive/1CN69NftfVXUliyv11FGbM7qOYbO0XON5)
3. [AutoML в Keras](https://colab.research.google.com/drive/1V7mfY8da0S-FbWxhQbchJM38JSJBmtoZ)
4. Домашняя работа

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [ ]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

In [ ]:
import keras_nlp as nlp

# Библиотека матричного вычисления
import numpy as np
# Библиотека для работы с данными
import pandas as pd
# Библиотека для работы с регулярными выражениями
import re
# Библиотека для работы с фреймворком TensorFlow
import tensorflow as tf
# Библиотека AutoML autokeras
import autokeras as ak
# Библиотеки для построения графиков и их стилизации
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Утилита для расщепления выборки
from sklearn.model_selection import train_test_split

# Необходимые метрики для построения Матрицы ошибок и отчета о классификации
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -qo "/content/drive/MyDrive/Consumer_Complaints.csv.zip" -d ./dataset

In [ ]:
address = "./dataset/Consumer_Complaints.csv"

In [ ]:
df = pd.read_csv(address)
df.head()

Датасет состоит из:

Date received — Дата получения\
Product — Продукт\
Sub-product — Подпродукт\
Issue — Проблема\
Sub-issue — Подпроблема\
Consumer complaint narrative — Текст жалобы потребителя\
Company public response — Публичный ответ компании\
Company — Компания\
State — Штат\
ZIP code — Почтовый индекс\
Tags — Метки\
Consumer consent provided? — Предоставлено согласие потребителя?\
Submitted via — Подано через\
Date sent to company — Дата отправки компании\
Company response to consumer — Ответ компании потребителю\
Timely response? — Своевременный ответ?\
Consumer disputed? — Оспорено потребителем?\
Complaint ID — Идентификатор жалобы

In [ ]:
# Удаление строк, где отсутствует текст жалобы потребителя или проблема
df = df.dropna(subset=["Consumer complaint narrative", "Issue"])

In [ ]:
# Очистка текста
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Удаляем переносы строк
    text = re.sub(r'\r', ' ', text)  # Удаляем возвраты каретки
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем пунктуацию
    text = re.sub(r'\d+', '', text)  # Удаляем цифры
    text = text.lower()  # Преобразуем текст к нижнему регистру
    return text

df["Consumer complaint narrative"] = df["Consumer complaint narrative"].apply(lambda x: clean_text(x) if isinstance(x, str) else x)
df["Issue"] = df["Issue"].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

In [ ]:
# Сохраним оригинальные метки
original_categories = df["Issue"].astype("category").cat.categories

In [ ]:
# Определим порог для минимального количества примеров в категории
min_samples = 2

# Найдем категории с малым количеством примеров
value_counts = df['Issue'].value_counts()
to_remove = value_counts[value_counts < min_samples].index

# Удалим строки с редкими категориями
df = df[~df['Issue'].isin(to_remove)]

In [ ]:
# Преобразование категорий в числовой формат
df['Issue'] = pd.Categorical(df.Issue.to_numpy())
df['Issue'] = df.Issue.cat.codes

In [ ]:
df.Issue.unique()

In [ ]:
# Обновляем переменные X и y
X = df["Consumer complaint narrative"].to_numpy()
y = df.Issue.to_numpy()

In [ ]:
# Разделение данных на обучающую, валидационную и тестовую выборки
X_tr, X_tmp, y_tr, y_tmp = train_test_split(X, y, stratify=y, test_size=0.3, random_state=15)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=15)

In [ ]:
print('Форма входных данных: ', X_tr.shape)
print('Форма выходных меток: ', y_tr.shape)
print('Пример заголовка: ', X_tr[0])

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextBlock()(input_node)
output_node = ak.ClassificationHead()(output_node)

clf_mod = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True,  max_trials=2, objective='val_accuracy'
)

res = clf_mod.fit(X_tr, y_tr, epochs=10)

In [ ]:
clf_mod.export_model().summary()

In [ ]:
preds = clf_mod.predict(X_test).astype(int)

print(clf_mod.evaluate(X_test, y_test))

In [ ]:
predicted_issues_text = pd.Categorical.from_codes(preds.flatten(), categories=original_categories)

In [ ]:
df_test = pd.DataFrame({'Consumer complaint narrative': X_test, 'Predicted Issue': predicted_issues_text})

In [ ]:
# Добавление предсказанных значений в DataFrame
df_test['Predicted Issue'] = preds

In [ ]:
df_test.head()

Итог: Точность модели составила 40%, учитывая сложность данных датасета, количество меток, а так же различный формат обращений